In [1]:
import os, sys, argparse, importlib, time, inspect
import numpy as np
import matplotlib.pyplot as plt
import os.path as osp
from tqdm.notebook import tqdm
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
import tensorflow_probability as tfp

gpu_devices = tf.config.list_physical_devices('GPU') 
if len(gpu_devices) > 0:
    print("GPU detected")
    for i in range(len(gpu_devices)):
        tf.config.experimental.set_memory_growth(gpu_devices[i], True)
else:
    print('No GPU detected')

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

import pandas as pd

GPU detected


In [144]:
dfjv=pd.read_csv('../predictions/long_3D.csv').sort_values('event_no')
dfjbs=pd.read_csv('../predictions/current_best_predictions.csv').sort_values('event_no')

In [145]:
x=pd.concat([dfjv['zenith_pred'], dfjv['angle_kap'], dfjv['energy_log10_pred'], dfjbs['zenith_pred'], dfjbs['zenith_kap'], dfjbs['energy_log10_pred']], axis=1)
y=dfjbs['zenith'].to_numpy()

In [ ]:
val=int(1e5)
hidden=64
model = tf.keras.Sequential()
for i in range(3):
    model.add(tf.keras.layers.Dense(hidden*i))
    model.add(BatchNormalization())
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Dense(1))
model.compile(optimizer='Adam', loss='mse', metrics='mae')
# This builds the model for the first time:
# model.fit(x[val:], y[val:], batch_size=1024, epochs=5)
model.fit(x, y, batch_size=1024, epochs=1)

model.summary()

In [ ]:
yreco=model.predict(x[:val])
ytrue=y[:val]

In [ ]:
res=(abs(yreco.T-ytrue)).T
plt.hist(res, bins=100);
plt.hist(abs(yreco.flatten()), bins=100);
plt.hist(np.mean(x[:val], axis=1), alpha=0.4, bins=100);
plt.hist(ytrue, bins=100, alpha=0.4);